In [ ]:
# !pip install -U bitsandbytes
# !pip install gTTS
# !pip install transformers datasets accelerate peft bitsandbytes

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

In [ ]:
# 경로 및 모델명
file_path = "/content/cleaned_dataset_half.json"
model_name = "beomi/KoAlpaca-Polyglot-5.8B"

# 1. 데이터 로드
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

dataset = Dataset.from_list(data)

In [ ]:
# 2. 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # LLaMA 계열에서 필수

# 3. 토크나이징 함수 정의
def tokenize(example):
    prompt = example["input"].strip()
    response = example["output"].strip()
    full_text = prompt + "\n" + response

    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize, remove_columns=["input", "output"])

In [ ]:
# 4. 양자화 설정 + 모델 로드
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=False  # CPU offload 금지
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

In [ ]:
# 5. LoRA 설정 및 적용
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["attention.query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)


model = get_peft_model(model, lora_config)


In [ ]:
# 6. 학습 설정
training_args = TrainingArguments(
    output_dir="/content/koalpaca-lora-checkpoints",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,
    report_to="none",
    fp16=True
)

In [ ]:
# 7. 데이터 콜레이터 및 Trainer
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# 8. 학습 실행
trainer.train()

# 9. 모델 저장
trainer.save_model("/content/beomi_llm")
tokenizer.save_pretrained("/content/beomi_llm")

In [ ]:
import shutil

source_dir = '/content/beomi_llm'
destination_dir = '/content/drive/MyDrive/디스부/예비 프로젝트/Beomi/beomi_llm'

shutil.copytree(source_dir, destination_dir)
